#### 자전거도로 제안
- 연도별 대여이력 정보를 불러와서 대여대여소, 반납대여소 두 값을 키로 보고 이용거리, 이용시간을 더하고, 갯수를 카운트하여 새로운 dataframe을 생성하여 저장
- 시각화

In [ ]:
%pip install folium

In [2]:
# 사용자 정의함수 경로 설정
import sys
sys.path.append("G:/내 드라이브/Source/_Custom_Function")
# 사용자 정의함수 불러오기
import ExcelFile_Merge as em
import ExcelFile_Read as er

import pandas as pd

#### 변수 설정

In [41]:
upto = 50000          # folium상 표현할 피처 개수
rush_hour = True    # 출퇴근시간 여부
start_year = 2023   # 시작연도  2018 ~ 2024
end_year = 2024     # 최종연도  2018 ~ 2024
show_data = False    # 데이터를 출력창에 display함
popup_info = False   # folium 상에서 마우스오버시 정보 팝업업
specific_rank = False
# specific_rank = [1, 4, 11, 20, 25, 35, 37, 45, 46]#[65, 68, 71, 79, 82, 88, 89, 90]
# [1, 4, 11, 13, 18, 20, 22]#, 25, 32, 35, 37, 43, 45, 46, 48, 57, 59, 65, 68, 71, 79, 82, 88, 89, 90]
#range(1, 101)
# #[2, 20, 33, 42, 49, 67, 69, 77]
#[69, 31, 32, 73, 12, 2, 6, 64, 77, 16, 22, 7, 11, 60, 35, 64, 27, 8, 20, 14, 67, 6, 42, 33, 49, 58, 73]         # 특정 이용량 순위의 길만 보고 싶을 때 입력 그 외는 빈 리스트로 두면 됨

#### station_info, 대여이력 정보에 위도, 경도 정보 병합

In [42]:
station_info = em.Read_File(r'G:\내 드라이브\DataSet\_파킷 파일\서울시 공공자전거 대여소 정보\공공자전거 대여소 정보_통합본_최종.parquet')

# 출퇴근시간 데이터 여부
if rush_hour:
    _rush_hour = '_평일출퇴근'
else:
    _rush_hour = ''

df = em.Read_File(r'G:\내 드라이브\DataSet\_파킷 파일\서울특별시 공공자전거 대여이력 정보\gis\공공자전거 대여반납 그룹_2018_2024' + _rush_hour + '.parquet')
df = df[(df['연도'] >= start_year) & (df['연도'] <= end_year)]
display(df.head(1))
display(df.tail(1))
df = pd.merge(df, station_info[['대여소번호', '위도', '경도']], left_on='대여대여소번호', right_on='대여소번호', how='left').rename(columns={'위도': '대여대여소위도', '경도': '대여대여소경도'})
df.drop('대여소번호', axis=1, inplace=True)
df = pd.merge(df, station_info[['대여소번호', '위도', '경도']], left_on='반납대여소번호', right_on='대여소번호', how='left').rename(columns={'위도': '반납대여소위도', '경도': '반납대여소경도'})
df.drop('대여소번호', axis=1, inplace=True)
display(df.head(1))
df = df[df['대여대여소번호'] != df['반납대여소번호']]
grouped_df = df.groupby(['대여대여소번호', '반납대여소번호', '연도']).agg({
    '이용시간(분)': 'sum',
    '이용거리(M)': 'sum',
    '이용량': 'sum',
    '대여대여소명': 'last',
    '대여상세주소': 'last',
    '반납대여소명': 'last',
    '반납상세주소': 'last',
    '대여대여소위도': 'last',
    '대여대여소경도': 'last',
    '반납대여소위도': 'last',
    '반납대여소경도': 'last'
    }).reset_index()
display(grouped_df.head(1))
display(grouped_df['연도'].unique())

,대여대여소번호,반납대여소번호,이용시간(분),이용거리(M),이용량,대여대여소명,대여상세주소,반납대여소명,반납상세주소,연도
1376721,00102,00005,11.0,2080.27,1,망원역 1번출구 앞,서울특별시 마포구 월드컵로 72,None,None,2023


,대여대여소번호,반납대여소번호,이용시간(분),이용거리(M),이용량,대여대여소명,대여상세주소,반납대여소명,반납상세주소,연도
2215766,09980,09980,0.0,0.0,1,None,None,None,None,2024


,대여대여소번호,반납대여소번호,이용시간(분),이용거리(M),이용량,대여대여소명,대여상세주소,반납대여소명,반납상세주소,연도,대여대여소위도,대여대여소경도,반납대여소위도,반납대여소경도
0,00102,00005,11.0,2080.27,1,망원역 1번출구 앞,서울특별시 마포구 월드컵로 72,None,None,2023,37.555649,126.910629,NaN,NaN


,대여대여소번호,반납대여소번호,연도,이용시간(분),이용거리(M),이용량,대여대여소명,대여상세주소,반납대여소명,반납상세주소,대여대여소위도,대여대여소경도,반납대여소위도,반납대여소경도
0,00102,00005,2023,11.0,2080.27,1,망원역 1번출구 앞,서울특별시 마포구 월드컵로 72,None,None,37.555649,126.910629,NaN,NaN


array([2023, 2024])

#### 연도별로 folium으로 시각화
- 총계 그룹화
- 자전거 도로 정보 추가

In [43]:
import json
import folium
import pandas as pd

# 지도 생성
center = [37.5665, 126.9780]
m = folium.Map(location=center, zoom_start=12)

# 데이터프레임, 지도에 표현할 연도 생성
grouped_df = grouped_df.dropna()
years = sorted(grouped_df['연도'].unique()) + [str(min(grouped_df['연도'].unique())) + '~' + str(max(grouped_df['연도'].unique()))]
feature_groups = {}

# 자전거 도로 데이터 로드
geo_path_road = r'G:\내 드라이브\DataSet\_최종 병합 파일\서울시 자전거 도로 데이터\ddareng_road.geojson'
with open(geo_path_road, 'rt', encoding='utf-8') as f:
    geo_str_road = json.load(f)


# 연도별 FeatureGroup 생성
for year in years:
    ##### 총계 그룹화
    if type(year) == str:
        df_year = grouped_df.groupby(['대여대여소번호', '반납대여소번호']).agg({
            '연도': lambda x: list(x),
            '이용시간(분)': 'sum',
            '이용거리(M)': 'sum',
            '이용량': 'sum',
            '대여대여소명': 'last',
            '대여상세주소': 'last',
            '반납대여소명': 'last',
            '반납상세주소': 'last',
            '대여대여소위도': 'last',
            '대여대여소경도': 'last',
            '반납대여소위도': 'last',
            '반납대여소경도': 'last'
            }).reset_index()
    else:
        df_year = grouped_df[grouped_df['연도'] == year]
    
    # df_year = df_year.sort_values(by='이용량', ascending=False).head(upto)
    df_year = df_year.sort_values(by='이용량', ascending=True).tail(upto)
    df_year['이용량순위'] = df_year['이용량'].rank(method="min", ascending=False).astype(int)

    if specific_rank:
        df_year = df_year[df_year['이용량순위'].isin(specific_rank)]

    if show_data:
        pd.options.display.max_rows = 100
        display(year)
        display(df_year.head(upto))
        display('*' * 150)
        pd.options.display.max_rows = 5
        
    # 데이터 생성
    data = {
        'start_lat': df_year['대여대여소위도'],
        'start_lon': df_year['대여대여소경도'],
        'end_lat': df_year['반납대여소위도'],
        'end_lon': df_year['반납대여소경도'],
        'usage': df_year['이용량'],  # 가중치
        'rank': df_year['이용량순위'],
        'rent_addr': df_year['대여상세주소'],
        'return_addr': df_year['반납상세주소']
    }
    df = pd.DataFrame(data).sort_values(by='usage', ascending=False)  # 이용량 기준 정렬

    df = pd.DataFrame(data)

    # df['usage_scaled'] = (df['usage'] - df['usage'].min()) / (df['usage'].max() - df['usage'].min())            # 사용량에 따른 색상 차이를 주기 위함
    # df['usage_scaled'] = (upto - df['rank']) / upto                                                                        # min-max scaling 기준을 이용량에서 이용량순위로 변경
    
    scaled_rank = (upto - df['rank']) / upto  # 0~1로 정규화
    df['usage_scaled'] = scaled_rank ** 15       ## 지수함수적으로 스케일링
    
    if specific_rank:
        df['usage_scaled'] = 1                                                                        # 특정 항목만 보고 싶을 때 스케일링 초기화
    
    # FeatureGroup 생성
    fg = folium.FeatureGroup(name=f"{year}년 이용량", show=False)
    
    # PolyLine 추가
    for _, row in df.iterrows():
        locations = [(row['start_lat'], row['start_lon']), (row['end_lat'], row['end_lon'])]
        weight = 1 + 2 * row['usage_scaled']       # 선의 두께

        red_intensity = int(255 * row['usage_scaled'])
        # blue_intensity = int(255 * (0.8 - 0.8 * row['usage_scaled']))
        blue_intensity = int(255 * (1 - row['usage_scaled']))
        color = f'#{red_intensity:02x}55{blue_intensity:02x}'

        # # # 형광연두색과 네이비색 간의 색상 조정
        # # red_intensity = int(50 * (1 - row['usage_scaled']))  # 적을수록 더 어두운 초록빛
        # # green_intensity = int(255 * row['usage_scaled'])     # 많을수록 형광연두색
        # # blue_intensity = int(128 * (1 - row['usage_scaled']))  # 적을수록 네이비색에 가까움
        # # # RGB를 Hex 색상으로 변환
        # # color = f'#{red_intensity:02x}{green_intensity:02x}{blue_intensity:02x}'

        # # 어두운 파란색에서 밝은 파란색으로 색상 조정
        # red_intensity = 0  # Red 값 고정
        # green_intensity = 0  # Green 값 고정
        # blue_intensity = int(50 + 205 * row['usage_scaled'])  # 적을수록 어두운 파란색, 많을수록 밝은 파란색

        # # RGB를 Hex 색상으로 변환
        # color = f'#{red_intensity:02x}{green_intensity:02x}{blue_intensity:02x}'


        opacity = 0.3 + 0.6 * row['usage_scaled']       # 선의 투명도
        
        # PolyLine에 Popup 추가하여 사용량 순위 및 사용량 표시
        # PolyLine에 Popup 추가
        line = folium.PolyLine(
            locations,
            color=color,
            weight=weight,
            opacity=opacity,
            smooth_factor=2.0,
            tooltip=f'Usage: {row["usage"]}<br>Usage Rank: {row["rank"]}<br>Rent Address: {row["rent_addr"]}<br>Return Address: {row["return_addr"]}'
            # tooltip=f'Usage: {row["usage"]}'
        )
        if popup_info:
            popup_text = f"Usage Rank: {row['rank']}<br>Usage: {row['usage']}"
            popup = folium.Popup(popup_text, max_width=250)

            line.add_child(popup)  # PolyLine에 Popup 추가                            용량을 많이 차지하므로 주석처리해도 됨
        line.add_to(fg)

    
    feature_groups[year] = fg
    m.add_child(fg)

fg0 = folium.FeatureGroup(name="bike_road", show=True).add_to(m)
# GeoJson 스타일 함수 정의
def _style_function(feature):
    return {
        'color': feature['properties'].get('LINE_COLOR', '#3388ff'),  # 기본 색상은 파란색
        #'weight': feature['properties'].get('LINE_WEIGHT', 1),       # 기본 굵기는 2
        'weight': 2,       # 기본 굵기는 2
        'opacity': 0.7
    }

# 자전거 도로 표시
# GeoJSON 파일을 불러와 스타일 적용
folium.GeoJson(
    geo_str_road,
    style_function=_style_function,
    name='bike_road'
).add_to(fg0)


# 자치구 구분 표
geo_str2 = json.load(open('G:\\내 드라이브\\DataSet\\seoul_municipalities_geo_simple.json', encoding='utf-8'))
choropleth = folium.Choropleth(geo_data = geo_str2,
                #   data = have_bikes,
                  name = 'gu',
                  key_on = 'feature.properties.SIG_KOR_NM',
                  fill_color = 'Blues',
                  fill_opacity=0.3              # 채우기 불투명도
                )
folium.GeoJson(
    geo_str2,
    style_function=lambda x: {
        'color': '#FFFFFF',  # Set boundary color to white
        'weight': 2,         # Boundary line thickness
        'opacity': 1         # Boundary line opacity
    }
).add_to(m)
folium.GeoJsonPopup(fields=['SIG_KOR_NM'],
                    aliases=['지역:'],
                    style='font-weight: bold;'
).add_to(choropleth.geojson)
choropleth.add_to(m)


# HTML 슬라이더 추가 및 JavaScript 코드 수정
m.get_root().html.add_child(folium.Element('''
    <div style="position: fixed; bottom: 10px; left: 50%; transform: translateX(-50%); z-index: 1000; background: white; padding: 10px; border-radius: 5px; width: 300px; text-align: center;">
        <label for="yearSlider" style="font-weight: bold;">연도 선택</label>
        <br>
        <input type="range" min="0" max="''' + str(len(years) - 1) + '''" value="0" id="yearSlider" oninput="updateMap(this.value)" style="width: 80%;">
        <br>
        <span id="yearLabel" style="display: inline-block; width: 150px; text-align: center;">''' + str(years[0]) + '''</span>
    </div>

    <script>
        let featureGroups = {};
        ''' + ''.join(f"featureGroups[{i}] = '{year}년 이용량';\n" for i, year in enumerate(years)) + '''
        
        function updateMap(value) {
            let selectedYear = featureGroups[value];
            document.getElementById('yearLabel').textContent = Object.values(featureGroups)[value];

            // 모든 레이어 숨기기
            for (let i = 0; i < Object.keys(featureGroups).length; i++) {
                let layerName = featureGroups[i];
                document.querySelectorAll(".leaflet-control-layers-selector").forEach((layer) => {
                    if (layer.nextSibling.textContent.trim() === layerName) {
                        layer.checked = (i == value);
                        layer.dispatchEvent(new Event("click"));
                    }
                });
            }
        }
    </script>
'''))

# Layer Control 추가
folium.TileLayer('cartodbpositron', name='CartoDB Positron').add_to(m)
folium.TileLayer('cartodbdark_matter', name='CartoDB Dark Matter').add_to(m)
folium.LayerControl().add_to(m)

# 지도 저장
m.save("bike_rent_return_map_" + str(start_year)[-2:] + "_" + str(end_year)[-2:] + "_upto_" + str(upto) + _rush_hour + "2.html")

#### 아래는 연습했던 코드

In [15]:
import folium

# 원하는 지점들의 좌표 정의
locations = [
    (37.5665, 126.9780),  # 서울
    (35.6895, 139.6917),  # 도쿄
    (40.7128, -74.0060),  # 뉴욕
    (51.5074, -0.1278)    # 런던
]

# folium 맵 생성
m = folium.Map(location=[0, 0], zoom_start=2)

# 각 지점에 마커 추가
for i, loc in enumerate(locations):
    folium.Marker(
        loc, 
        popup=f'Location {i+1}',
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)

# JavaScript 코드 작성
js = """
<script>
var locations = %s;
var index = 0;
var map = null;

function initMap(mapObject) {
    map = mapObject;
    setTimeout(zoomToNext, 1000);
}

function zoomToNext() {
    if (index < locations.length) {
        var loc = locations[index];
        map.flyTo(loc, 10, {
            duration: 2
        });
        index++;
        setTimeout(zoomToNext, 5000);  // 5초 후 다음 위치로 이동
    }
}

window.addEventListener('load', function() {
    initMap(document.querySelector('.folium-map')._leaflet_map);
});
</script>
""" % str(locations)

# JavaScript 코드를 맵에 추가
m.get_root().html.add_child(folium.Element(js))

# 맵 저장
m.save("sequential_zoom_map.html")

In [21]:
# import folium
# import pandas as pd

# # 지도 생성
# center = [37.5665, 126.9780]
# m = folium.Map(location=center, zoom_start=13)

# # 예시 데이터프레임 준비
# grouped_df = grouped_df.dropna()
# years = sorted(grouped_df['연도'].unique()) + [str(min(grouped_df['연도'].unique())) + '~' + str(max(grouped_df['연도'].unique()))]
# feature_groups = {}

# # 연도별 FeatureGroup 생성
# for year in years:
#     ##### 총계 그룹화
#     if type(year) == str:
#         df_year = grouped_df.groupby(['대여대여소번호', '반납대여소번호']).agg({
#             '연도': lambda x: list(x),
#             '이용시간(분)': 'sum',
#             '이용거리(M)': 'sum',
#             '이용량': 'sum',
#             '대여대여소명': 'last',
#             '대여상세주소': 'last',
#             '반납대여소명': 'last',
#             '반납상세주소': 'last',
#             '대여대여소위도': 'last',
#             '대여대여소경도': 'last',
#             '반납대여소위도': 'last',
#             '반납대여소경도': 'last'
#             }).reset_index()
#     else:
#         df_year = grouped_df[grouped_df['연도'] == year]
    
#     df_year = df_year.sort_values(by='이용량', ascending=False).head(upto)
#     df_year['이용량순위'] = df_year['이용량'].rank(method="min", ascending=False).astype(int)

#     # 데이터 생성
#     data = {
#         'start_lat': df_year['대여대여소위도'],
#         'start_lon': df_year['대여대여소경도'],
#         'end_lat': df_year['반납대여소위도'],
#         'end_lon': df_year['반납대여소경도'],
#         'usage': df_year['이용량'],  # 가중치
#         'rank': df_year['이용량순위']
#     }
#     df = pd.DataFrame(data)
#     df['usage_scaled'] = (df['usage'] - df['usage'].min()) / (df['usage'].max() - df['usage'].min())
    
#     # FeatureGroup 생성
#     fg = folium.FeatureGroup(name=f"{year}년 이용량", show=False)
    
#     # PolyLine 추가
#     for _, row in df.iterrows():
#         locations = [(row['start_lat'], row['start_lon']), (row['end_lat'], row['end_lon'])]
#         weight = 1 + 10 * row['usage_scaled']
#         red_intensity = int(255 * row['usage_scaled'])
#         blue_intensity = int(255 * (1 - row['usage_scaled']))
#         color = f'#{red_intensity:02x}55{blue_intensity:02x}'
#         opacity = 0.2 + 0.6 * row['usage_scaled']
        
#         # PolyLine에 Popup 추가하여 사용량 순위 및 사용량 표시                      용량을 많이 차지하므로 주석처리해도 됨
#         popup_text = f"Usage Rank: {row['rank']}<br>Usage: {row['usage']}"
#         popup = folium.Popup(popup_text, max_width=250)

#         # PolyLine에 Popup 추가
#         line = folium.PolyLine(
#             locations,
#             color=color,
#             weight=weight,
#             opacity=opacity,
#             smooth_factor=2.0,
#             tooltip=f'Usage: {row["usage"]}'
#         )
#         line.add_child(popup)  # PolyLine에 Popup 추가                            용량을 많이 차지하므로 주석처리해도 됨
#         line.add_to(fg)

    
#     feature_groups[year] = fg
#     m.add_child(fg)

# # HTML 슬라이더 추가 및 JavaScript 코드 수정
# m.get_root().html.add_child(folium.Element('''
#     <div style="position: fixed; bottom: 10px; left: 50%; transform: translateX(-50%); z-index: 1000; background: white; padding: 10px; border-radius: 5px; width: 300px; text-align: center;">
#         <label for="yearSlider" style="font-weight: bold;">연도 선택</label>
#         <br>
#         <input type="range" min="0" max="''' + str(len(years) - 1) + '''" value="0" id="yearSlider" oninput="updateMap(this.value)" style="width: 80%;">
#         <br>
#         <span id="yearLabel" style="display: inline-block; width: 150px; text-align: center;">''' + str(years[0]) + '''</span>
#     </div>

#     <script>
#         let featureGroups = {};
#         ''' + ''.join(f"featureGroups[{i}] = '{year}년 이용량';\n" for i, year in enumerate(years)) + '''
        
#         function updateMap(value) {
#             let selectedYear = featureGroups[value];
#             document.getElementById('yearLabel').textContent = Object.values(featureGroups)[value];

#             // 모든 레이어 숨기기
#             for (let i = 0; i < Object.keys(featureGroups).length; i++) {
#                 let layerName = featureGroups[i];
#                 document.querySelectorAll(".leaflet-control-layers-selector").forEach((layer) => {
#                     if (layer.nextSibling.textContent.trim() === layerName) {
#                         layer.checked = (i == value);
#                         layer.dispatchEvent(new Event("click"));
#                     }
#                 });
#             }
#         }
#     </script>
# '''))

# # Layer Control 추가
# folium.TileLayer('cartodbpositron', name='CartoDB Positron').add_to(m)
# folium.TileLayer('cartodbdark_matter', name='CartoDB Dark Matter').add_to(m)
# folium.LayerControl().add_to(m)

# # 지도 저장
# m.save("bike_rent_return_map_slider_upto_" + str(upto) + _rush_hour + ".html")


In [22]:
# import folium
# import pandas as pd

# # 예시 데이터프레임 생성
# grouped_df = grouped_df.dropna()

# # Folium 지도 생성 (서울을 중심으로 설정)
# m = folium.Map(location=[37.5665, 126.9780], zoom_start=13)

# # 연도별로 데이터프레임을 필터링하고 각 연도에 해당하는 레이어 그룹 생성
# years = sorted(grouped_df['연도'].unique()) + ['총계']
# for year in years:
#     # 해당 연도 데이터 필터링
#     if year == '총계':
#         df_year = grouped_df.groupby(['대여대여소번호', '반납대여소번호']).agg({
#             '연도': lambda x: list(x),
#             '이용시간(분)': 'sum',
#             '이용거리(M)': 'sum',
#             '이용량': 'sum',
#             '대여대여소명': 'last',
#             '대여상세주소': 'last',
#             '반납대여소명': 'last',
#             '반납상세주소': 'last',
#             '대여대여소위도': 'last',
#             '대여대여소경도': 'last',
#             '반납대여소위도': 'last',
#             '반납대여소경도': 'last'
#             }).reset_index()
#     else:
#         df_year = grouped_df[grouped_df['연도'] == year]
    
#     df_year = df_year.sort_values(by='이용량', ascending=False).head(10000)

#     # 연도별로 데이터프레임 생성
#     data = {
#         'start_lat': df_year['대여대여소위도'],
#         'start_lon': df_year['대여대여소경도'],
#         'end_lat': df_year['반납대여소위도'],
#         'end_lon': df_year['반납대여소경도'],
#         'usage': df_year['이용량']  # 가중치
#     }
#     df = pd.DataFrame(data)
    
#     # Min-Max Scaling 적용
#     df['usage_scaled'] = (df['usage'] - df['usage'].min()) / (df['usage'].max() - df['usage'].min())
    
#     # 사용량이 많은 순으로 정렬하여 높은 사용량의 선이 위에 그려지도록 조정
#     df = df.sort_values(by='usage_scaled', ascending=True)
    
#     # 연도별로 레이어 그룹 생성
#     fg = folium.FeatureGroup(name=f"{year}년 이용량").add_to(m)
    
#     # 출발지-도착지 선 추가
#     for _, row in df.iterrows():
#         # 좌표 리스트 정의
#         locations = [(row['start_lat'], row['start_lon']), (row['end_lat'], row['end_lon'])]
        
#         # 가중치에 따른 밝기 및 선의 두께와 색상, 투명도 설정
#         weight = 1 + 10 * row['usage_scaled']  # 기본 두께 1, 최대 두께 11
#         red_intensity = int(255 * row['usage_scaled'])  # 빨간색 강도는 사용량에 비례
#         blue_intensity = int(255 * (1 - row['usage_scaled']))  # 파란색 강도는 사용량에 반비례
#         color = f'#{red_intensity:02x}55{blue_intensity:02x}'  # 초록빛을 더하여 그라데이션 효과
        
#         # 투명도 설정
#         opacity = 0.2 + 0.6 * row['usage_scaled']  # 적은 이용량은 투명도 0.2, 많은 이용량은 0.8

#         # PolyLine 추가 (부드러운 선과 그라데이션)
#         folium.PolyLine(
#             locations,
#             color=color,
#             weight=weight,
#             opacity=opacity,
#             smooth_factor=2.0,  # 부드럽게 선을 표현
#             tooltip=f'Usage: {row["usage"]}'
#         ).add_to(fg)

# # 레이어 컨트롤 추가
# folium.TileLayer('cartodbpositron', name='CartoDB Positron').add_to(m)
# folium.TileLayer('cartodbdark_matter', name='CartoDB Dark Matter').add_to(m)
# folium.LayerControl().add_to(m)

# # 지도 저장 및 출력
# m.save("bike_rent_return_map_yearly.html")


In [ ]:
# import folium
# import pandas as pd

# # 예시 데이터프레임 준비: 출발지 위도, 출발지 경도, 도착지 위도, 도착지 경도, 가중치(사용량)
# grouped_df = grouped_df.dropna()
# grouped_df = grouped_df.sort_values(by='이용량', ascending=False).head(10000)

# # 데이터프레임 생성
# data = {
#     'start_lat': grouped_df['대여대여소위도'],
#     'start_lon': grouped_df['대여대여소경도'],
#     'end_lat': grouped_df['반납대여소위도'],
#     'end_lon': grouped_df['반납대여소경도'],
#     'usage': grouped_df['이용량']  # 가중치
# }
# df = pd.DataFrame(data)

# # Min-Max Scaling 적용
# df['usage_scaled'] = (df['usage'] - df['usage'].min()) / (df['usage'].max() - df['usage'].min())

# # 사용량이 많은 순으로 정렬하여 높은 사용량의 선이 위에 그려지도록 조정
# df = df.sort_values(by='usage_scaled', ascending=True)

# # Folium 지도 생성 (서울을 중심으로 설정)
# m = folium.Map(location=[37.5665, 126.9780], zoom_start=13)

# # 출발지-도착지 선 추가
# for _, row in df.iterrows():
#     # 좌표 리스트 정의
#     locations = [(row['start_lat'], row['start_lon']), (row['end_lat'], row['end_lon'])]
    
#     # 가중치에 따른 밝기 및 선의 두께와 색상, 투명도 설정
#     weight = 1 + 10 * row['usage_scaled']  # 기본 두께 1, 최대 두께 11
#     red_intensity = int(255 * row['usage_scaled'])  # 빨간색 강도는 사용량에 비례
#     blue_intensity = int(255 * (1 - row['usage_scaled']))  # 파란색 강도는 사용량에 반비례
#     color = f'#{red_intensity:02x}55{blue_intensity:02x}'  # 초록빛을 더하여 그라데이션 효과
    
#     # 투명도 설정
#     opacity = 0.2 + 0.6 * row['usage_scaled']  # 적은 이용량은 투명도 0.2, 많은 이용량은 0.8

#     # PolyLine 추가 (부드러운 선과 그라데이션)
#     folium.PolyLine(
#         locations,
#         color=color,
#         weight=weight,
#         opacity=opacity,
#         smooth_factor=2.0,  # 부드럽게 선을 표현
#         tooltip=f'Usage: {row["usage"]}'
#     ).add_to(m)

# # 결과 지도 출력
# m

# m.save("bike_rent_return_map.html")


In [ ]:
######### 용량이 너무 커서 느림
# import folium
# import pandas as pd
# import numpy as np

# # 예시 데이터프레임 준비: 출발지 위도, 출발지 경도, 도착지 위도 경도, 사용량(가중치)
# grouped_df = grouped_df.dropna()
# grouped_df = grouped_df.sort_values(by='이용량', ascending=False).head(10000)

# data = {
#     'start_lat': grouped_df['대여대여소위도'],
#     'start_lon': grouped_df['대여대여소경도'],
#     'end_lat': grouped_df['반납대여소위도'],
#     'end_lon': grouped_df['반납대여소경도'],
#     'usage': grouped_df['이용량']  # 가중치
# }
# df = pd.DataFrame(data)

# # Min-Max Scaling 적용
# df['usage_scaled'] = (df['usage'] - df['usage'].min()) / (df['usage'].max() - df['usage'].min())

# # 사용량이 많은 순으로 정렬하여 높은 사용량의 선이 위에 그려지도록 조정
# df = df.sort_values(by='usage_scaled', ascending=True)

# # Folium 지도 생성 (서울을 중심으로 설정)
# m = folium.Map(location=[37.5665, 126.9780], zoom_start=13)

# # 출발지-도착지 선 추가
# num_segments = 20  # 그라데이션 효과를 위해 분할할 구간 수
# for _, row in df.iterrows():
#     # 각 segment의 좌표 계산
#     latitudes = np.linspace(row['start_lat'], row['end_lat'], num_segments + 1)
#     longitudes = np.linspace(row['start_lon'], row['end_lon'], num_segments + 1)
    
#     # 가중치에 따른 색상 설정
#     weight = 2 + 12 * row['usage_scaled']  # 기본 두께 2, 최대 두께 14
#     red_intensity = int(255 * row['usage_scaled'])  # 빨간색 강도는 사용량에 비례
#     blue_intensity = int(255 * (1 - row['usage_scaled']))  # 파란색 강도는 사용량에 반비례
#     color = f'#{red_intensity:02x}55{blue_intensity:02x}'

#     # 각 segment마다 선을 그려서 점진적인 투명도 적용
#     for i in range(num_segments):
#         opacity = 0.8 * (1 - (i / num_segments))  # 점진적으로 투명도 감소
#         segment = [(latitudes[i], longitudes[i]), (latitudes[i + 1], longitudes[i + 1])]
#         folium.PolyLine(
#             segment,
#             color=color,
#             weight=weight,
#             opacity=opacity,
#             smooth_factor=2.0
#         ).add_to(m)

# # 결과 지도 출력
# m


In [ ]:
# import folium
# import pandas as pd

# # 예시 데이터프레임: 출발지 위도, 출발지 경도, 도착지 위도, 도착지 경도, 가중치(사용량)
# grouped_df = grouped_df.dropna()
# grouped_df = grouped_df.sort_values(by='이용량', ascending=False).head(10000)


# data = {
#     'start_lat': grouped_df['대여대여소위도'],
#     'start_lon': grouped_df['대여대여소경도'],
#     'end_lat': grouped_df['반납대여소위도'],
#     'end_lon': grouped_df['반납대여소경도'],
#     'usage': grouped_df['이용량']  # 가중치
# }

# # 데이터프레임 생성
# df = pd.DataFrame(data)

# # Min-Max Scaling 적용
# df['usage_scaled'] = (df['usage'] - df['usage'].min()) / (df['usage'].max() - df['usage'].min())

# # 사용량이 많은 순으로 정렬하여 높은 사용량의 선이 위에 그려지도록 조정
# df = df.sort_values(by='usage_scaled', ascending=True)

# # Folium 지도 생성 (서울을 중심으로 설정)
# m = folium.Map(location=[37.5665, 126.9780], zoom_start=13)

# # 출발지-도착지 선 추가
# for _, row in df.iterrows():
#     # 좌표 리스트 정의
#     locations = [(row['start_lat'], row['start_lon']), (row['end_lat'], row['end_lon'])]
    
#     # 가중치에 따른 밝기 및 선의 두께 설정
#     weight = 2 + 12 * row['usage_scaled']  # 기본 두께 2, 최대 두께 14
#     red_intensity = int(255 * row['usage_scaled'])  # 빨간색 강도는 이용량에 비례
#     blue_intensity = int(255 * (1 - row['usage_scaled']))  # 파란색 강도는 이용량에 반비례
#     color = f'#{red_intensity:02x}00{blue_intensity:02x}'  # 적은 쪽 파란색, 많은 쪽 빨간색
    
#     # PolyLine 추가 (빛나는 선 모양)
#     folium.PolyLine(
#         locations,
#         color=color,
#         weight=weight,
#         opacity=0.7,
#         tooltip=f'Usage: {row["usage"]}'
#     ).add_to(m)

# # 결과 지도 출력
# m


In [ ]:
%pip install folium